# SUUMO データ基本分析

このノートブックでは、スクレイピングしたSUUMOデータの基本的な分析を行います。

## 目次
1. データの読み込みと確認
2. 基本統計情報
3. 駅別分析
4. 間取り別分析
5. 築年数と賃料の関係
6. お得物件の発見

In [ ]:
# 必要なライブラリのインポート
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from analyzer import SuumoAnalyzer
from visualizer import SuumoVisualizer

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

# 日本語フォント設定
plt.rcParams['font.family'] = ['DejaVu Sans', 'Hiragino Sans', 'Yu Gothic', 'Meiryo']

print("📊 SUUMO データ分析ノートブック")
print("===============================\n")

## 1. データの読み込みと確認

In [ ]:
# データの読み込み
print("🔄 データを読み込み中...")
analyzer = SuumoAnalyzer(data_path="../../data", auto_load=True)

if analyzer.df is not None:
    print(f"✅ データ読み込み完了: {len(analyzer.df)} 件")
    print(f"📅 期間: {analyzer.df['scraped_at'].min()} 〜 {analyzer.df['scraped_at'].max()}")
else:
    print("❌ データの読み込みに失敗しました")

In [ ]:
# データの基本情報
print("📋 データ概要")
print("==============")
print(f"行数: {len(analyzer.df):,}")
print(f"列数: {len(analyzer.df.columns)}")
print(f"\n📊 カラム情報:")
analyzer.df.info()

In [ ]:
# 最初の数行を確認
print("👀 データサンプル:")
analyzer.df.head()

## 2. 基本統計情報

In [ ]:
# 基本統計情報の取得
basic_stats = analyzer.get_basic_stats()

print("📈 基本統計情報")
print("================\n")

print(f"🏠 総物件数: {basic_stats['total_properties']:,} 件")
print(f"🚉 対象駅数: {basic_stats['stations']['count']} 駅")
print(f"📍 対象駅: {', '.join(basic_stats['stations']['list'])}")

if 'rent' in basic_stats:
    rent_stats = basic_stats['rent']
    print(f"\n💰 賃料統計 (n={rent_stats['count']:,})")
    print(f"   平均: {rent_stats['mean']:,.0f} 円")
    print(f"   中央値: {rent_stats['median']:,.0f} 円")
    print(f"   最小: {rent_stats['min']:,.0f} 円")
    print(f"   最大: {rent_stats['max']:,.0f} 円")
    print(f"   25%: {rent_stats['q25']:,.0f} 円")
    print(f"   75%: {rent_stats['q75']:,.0f} 円")

if 'area' in basic_stats:
    area_stats = basic_stats['area']
    print(f"\n📐 面積統計 (n={area_stats['count']:,})")
    print(f"   平均: {area_stats['mean']:.1f} ㎡")
    print(f"   中央値: {area_stats['median']:.1f} ㎡")
    print(f"   最小: {area_stats['min']:.1f} ㎡")
    print(f"   最大: {area_stats['max']:.1f} ㎡")

if 'building_age' in basic_stats:
    age_stats = basic_stats['building_age']
    print(f"\n🏗️ 築年数統計 (n={age_stats['count']:,})")
    print(f"   平均: {age_stats['mean']:.1f} 年")
    print(f"   中央値: {age_stats['median']:.1f} 年")
    print(f"   最小: {age_stats['min']:.0f} 年")
    print(f"   最大: {age_stats['max']:.0f} 年")

In [ ]:
# 数値データの詳細統計
numeric_columns = ['rent_numeric', 'area_numeric', 'building_age', 'rent_per_sqm']
available_numeric = [col for col in numeric_columns if col in analyzer.df.columns]

if available_numeric:
    print("📊 数値データ統計:")
    analyzer.df[available_numeric].describe()

## 3. 駅別分析

In [ ]:
# 駅別分析
try:
    station_analysis = analyzer.analyze_rent_by_station()
    print("🚉 駅別賃料分析")
    print("================\n")
    
    # 上位5駅
    print("💰 賃料が高い駅 TOP5:")
    top_stations = station_analysis.nlargest(5, 'rent_numeric_mean')
    for i, row in top_stations.iterrows():
        print(f"{row.name+1:2d}. {row['search_station']}: {row['rent_numeric_mean']:,.0f}円 (n={row['rent_numeric_count']})")
    
    print("\n💸 賃料が安い駅 TOP5:")
    bottom_stations = station_analysis.nsmallest(5, 'rent_numeric_mean')
    for i, row in bottom_stations.iterrows():
        print(f"{i+1:2d}. {row['search_station']}: {row['rent_numeric_mean']:,.0f}円 (n={row['rent_numeric_count']})")
    
    print("\n📋 全駅統計:")
    display(station_analysis)
    
except Exception as e:
    print(f"❌ 駅別分析でエラー: {e}")

## 4. 間取り別分析

In [ ]:
# 間取り別分析
try:
    layout_analysis = analyzer.analyze_layout_distribution()
    print("🏠 間取り別分析")
    print("================\n")
    
    print("📊 間取り分布:")
    for i, row in layout_analysis.iterrows():
        print(f"{row['layout']}: {row['rent_numeric_count']}件 ({row['percentage']:.1f}%) - 平均{row['rent_numeric_mean']:,.0f}円")
    
    print("\n📋 詳細統計:")
    display(layout_analysis)
    
except Exception as e:
    print(f"❌ 間取り分析でエラー: {e}")

## 5. 築年数と賃料の関係

In [ ]:
# 築年数分析
try:
    age_analysis = analyzer.analyze_age_impact()
    print("🏗️ 築年数と賃料の関係")
    print("======================\n")
    
    print("📊 築年数カテゴリ別統計:")
    for i, row in age_analysis.iterrows():
        rent_change = f"({row['rent_vs_newest']:+.1f}%)" if 'rent_vs_newest' in age_analysis.columns else ""
        print(f"{row['age_category']}: {row['rent_numeric_count']}件 - 平均{row['rent_numeric_mean']:,.0f}円 {rent_change}")
    
    print("\n📋 詳細統計:")
    display(age_analysis)
    
except Exception as e:
    print(f"❌ 築年数分析でエラー: {e}")

## 6. お得物件の発見

In [ ]:
# お得物件分析
try:
    deals = analyzer.find_deals(
        rent_percentile=30,  # 賃料下位30%
        area_percentile=70,  # 面積上位30%
        max_age=20           # 築20年以内
    )
    
    print("💎 お得物件分析")
    print("================\n")
    print(f"🎯 検索条件:")
    print(f"   - 賃料: 下位30%以下")
    print(f"   - 面積: 上位30%以上")
    print(f"   - 築年数: 20年以内")
    
    if len(deals) > 0:
        print(f"\n✨ 発見されたお得物件: {len(deals)}件")
        
        print("\n🏆 お得度TOP10:")
        top_deals = deals.head(10)
        
        display_columns = ['building_title', 'search_station', 'rent_numeric', 'area_numeric', 'building_age', 'layout']
        available_columns = [col for col in display_columns if col in deals.columns]
        
        if 'deal_score' in deals.columns:
            available_columns.append('deal_score')
        
        display(top_deals[available_columns])
    else:
        print("\n😔 条件に合うお得物件は見つかりませんでした")
        
except Exception as e:
    print(f"❌ お得物件分析でエラー: {e}")

## 7. 総合レポート生成

In [ ]:
# 総合分析レポート
try:
    print("📋 総合分析レポート生成中...")
    summary_report = analyzer.generate_summary_report()
    
    print("\n🔍 主要インサイト:")
    print("==================")
    
    for i, insight in enumerate(summary_report.get('insights', []), 1):
        print(f"{i}. {insight}")
    
    # レポートを保存
    report_path = analyzer.export_analysis_results()
    print(f"\n💾 分析結果を保存しました: {report_path}")
    
except Exception as e:
    print(f"❌ レポート生成でエラー: {e}")

## 次のステップ

この基本分析の結果を踏まえて、以下の分析も実行できます：

1. **詳細可視化**: `02_visualization.ipynb` で各種グラフとチャートを生成
2. **比較分析**: `03_station_comparison.ipynb` で特定駅間の詳細比較
3. **予測モデル**: `04_price_prediction.ipynb` で賃料予測モデルの構築

これらのノートブックを順次実行して、より詳細な分析を行ってください。